# Introduction

This notebook illustrates the basics of how to use `plottr` -- in particular, the `inspectr` and `autoplot` tools -- to live plot data in a qcodes database.

## Basic notebook setup

In [ ]:
%gui qt

DBPATH = './qcodes_liveplot_demo.db'

import qcodes as qc

qc.config.core.db_location = DBPATH
qc.initialise_database()

# Launching inspectr

Next, we need to run the inspectr tool from the command line in a separate process. From within the plottr root directory, run 

``
$ python apps/inspectr.py --dbpath=./doc/examples/qcodes_liveplot_demo.db
``

We should now have two windows open; no data is yet shown if we started with a fresh .db file. 
Now, before populating the database, let's enable automatic monitoring of the dataset. To do that, enter a refresh interval (given in seconds) in the inspectr window toolbar, and enable the auto-plot option.

# Dummy experiments

Below are a few dummy qcodes experiments that should hopefully illustrate how the live plotter behaves. Run them while the inspectr is open, and monitoring is active (or not -- you can also refresh manually by pressing 'R'; this works for both inspectr and the autoplotter). 

## Qcodes imports (and other relevant stuff)

In [ ]:
import time
import numpy as np
import lmfit
from qcodes import load_or_create_experiment, Measurement, Parameter

## A very simple 1D sweep

In [ ]:
xvals = np.linspace(0, 10, 101)
yvals = np.sin(xvals)
y2vals = np.cos(xvals)

def simple_1d_sweep():
    for x, y, y2 in zip(xvals, yvals, y2vals):
        yield x, y, y2
        
x = Parameter('x')
y = Parameter('y')
y2 = Parameter('y2')

station = qc.Station(x, y, y2)

In [ ]:
exp = load_or_create_experiment('simple_1d_sweep', sample_name='no sample')

meas = Measurement(exp, station)
meas.register_parameter(x)
meas.register_parameter(y, setpoints=(x,))
meas.register_parameter(y2, setpoints=(x,))
meas.write_period = 2

with meas.run() as datasaver:
    for xval, yval, y2val in simple_1d_sweep():
        datasaver.add_result(
            (x, xval),
            (y, yval),
            (y2, y2val),
        )
        time.sleep(0.2)

## A simple 2D sweep

In [ ]:
xvals = np.linspace(-5, 5, 51)
yvals = np.linspace(-5, 5, 51)
xx, yy = np.meshgrid(xvals, yvals, indexing='ij')
zz = np.cos(xx) * np.cos(yy)

def simple_2d_sweep():
    for i, x in enumerate(xvals):
        yield x, yy[i, :], zz[i, :]
        
x = Parameter('x')
y = Parameter('y')
z = Parameter('z')

station = qc.Station(x, y, z)

In [ ]:
exp = load_or_create_experiment('simple_2d_sweep', sample_name='no sample')

meas = Measurement(exp, station)
meas.register_parameter(x)
meas.register_parameter(y, paramtype='array')
meas.register_parameter(z, setpoints=(x, y), paramtype='array')
meas.write_period = 2

with meas.run() as datasaver:
    for xval, yval, zval in simple_2d_sweep():
        datasaver.add_result(
            (x, xval),
            (y, yval),
            (z, zval),
        )
        time.sleep(0.2)

## Another simple 2D sweep

This is handy to use with the autonode examples. Autonode is a deocorator that enables fairly easy creation of new processing tools ('nodes') that can be integrated into plottr. As an example, we look at a simple node that just fits a 1d data set with a sine function. It's only user input is the initial guess of the frequency.
For the implementation, see the ``autonode_app.py`` file in the ``doc/examples`` folder.
(Note: you need `lmfit` installed for this one).

To see what it does, start the example below -- it's purposefully set to a very slow repetition rate. Then note down the runId that's displayed, and from this directory run:

``
$ python autonode.py ./qcodes_liveplot_demo.db <runid>
``

You can of course kill it anytime you're getting bored.

In [ ]:
# set up dummy data
xvals = np.linspace(-5, 5, 51)
yvals = np.arange(100.)
xx, yy = np.meshgrid(xvals, yvals, indexing='ij')
zz = np.sin(xx) + np.random.normal(scale=1, size=xx.shape)

def simple_2d_sweep():
    for i, y in enumerate(yvals):
        yield xx[:, i], y, zz[:, i]
        
x = Parameter('x')
y = Parameter('y')
z = Parameter('z')
station = qc.Station(x, y, z)

# ... and run
exp = load_or_create_experiment('another_simple_2d_sweep', sample_name='no sample')

meas = Measurement(exp, station)
meas.register_parameter(x, paramtype='array')
meas.register_parameter(y)
meas.register_parameter(z, setpoints=(x, y), paramtype='array')
meas.write_period = 2

with meas.run() as datasaver:
    for xval, yval, zval in simple_2d_sweep():
        datasaver.add_result(
            (x, xval),
            (y, yval),
            (z, zval),
        )
        time.sleep(2)